In [71]:
from util import *
from glob import glob
import matplotlib.pyplot as plt
from shapely import wkt
pd.set_option("display.max_columns", None)

In [72]:
gdf = gpd.read_file(f"TEST/Auckland_LongBay/LongBay_intersects.shp")
gdf["Date"] = pd.to_datetime(gdf.ShorelineI, dayfirst=True, format='mixed')
gdf["Year"] = gdf.Date.dt.year
gdf["YearsSinceBase"] = (gdf.Date - pd.Timestamp(1800, 1, 1)).dt.days / 365.25
gdf["YearsUntilFuture"] = (
    pd.Timestamp(2100, 1, 1) - gdf.Date
    ).dt.days / 365.25
gdf.Date = gdf.Date.astype(str)
gdf

#Why are distances negative? Because baseline is now offshore i.e zero is in the ocean?
#Do we need to ignore the sign? Erosion projection needs to show more negative distance (move onshore), while accretion needs to go in less negative (move offshore)

,TransectID,ShorelineI,BaselineID,Distance,IntersectX,IntersectY,Uncertaint,geometry,Date,Year,YearsSinceBase,YearsUntilFuture
0,1.0,02/25/2023,1.0,-40.88,1.756545e+06,5.938224e+06,2.30,POINT Z (1756544.660 5938224.155 0.000),2023-02-25,2023,223.145791,76.848734
1,1.0,01/19/1973,1.0,-98.70,1.756491e+06,5.938204e+06,3.76,POINT Z (1756490.537 5938203.811 0.000),1973-01-19,1973,173.045859,126.948665
2,1.0,01/11/2023,1.0,-38.31,1.756547e+06,5.938225e+06,2.30,POINT Z (1756547.060 5938225.058 0.000),2023-11-01,2023,223.827515,76.167009
3,1.0,06/20/2011,1.0,-28.21,1.756557e+06,5.938229e+06,0.97,POINT Z (1756556.514 5938228.611 0.000),2011-06-20,2011,211.460643,88.533881
4,1.0,12/28/2022,1.0,-38.67,1.756547e+06,5.938225e+06,2.21,POINT Z (1756546.722 5938224.931 0.000),2022-12-28,2022,222.984257,77.010267
...,...,...,...,...,...,...,...,...,...,...,...,...
1022,105.0,06/20/2011,1.0,-13.56,1.756192e+06,5.939182e+06,0.97,POINT Z (1756191.682 5939181.798 0.000),2011-06-20,2011,211.460643,88.533881
1023,105.0,12/28/2022,1.0,-25.95,1.756186e+06,5.939171e+06,2.21,POINT Z (1756186.425 5939170.582 0.000),2022-12-28,2022,222.984257,77.010267
1024,105.0,05/02/2017,1.0,-19.09,1.756189e+06,5.939177e+06,0.73,POINT Z (1756189.337 5939176.796 0.000),2017-02-05,2017,217.092402,82.902122
1025,105.0,03/29/1981,1.0,-24.60,1.756187e+06,5.939172e+06,4.73,POINT Z (1756186.996 5939171.801 0.000),1981-03-29,1981,181.234771,118.759754


In [73]:
lines = gpd.read_file("TEST/Auckland_LongBay/LongBay_rates.shp")
lines

,ObjectID_1,BaselineID,TransOrder,TransEdit,transectId,ShrCount,LRR,LR2,LCI,LSE,EPR,EPRunc,NSM,SCE,WLR,WR2,WCI,WSE,TCD,Azimuth,geometry
0,1.0,1.0,1.0,0.0,1.0,7.0,1.14,0.90,0.34,10.54,0.96,0.07,57.64,70.49,1.01,0.60,0.74,7.08,0.0,249.399436,"LINESTRING Z (1756582.920 5938238.537 0.000, 1..."
1,2.0,1.0,2.0,0.0,2.0,8.0,1.10,0.92,0.26,10.67,0.88,0.06,63.77,73.77,1.07,0.78,0.45,5.98,10.0,249.399436,"LINESTRING Z (1756579.402 5938247.898 0.000, 1..."
2,3.0,1.0,3.0,0.0,3.0,8.0,1.15,0.95,0.21,8.61,0.97,0.06,70.11,74.41,1.12,0.86,0.37,4.82,20.0,249.399436,"LINESTRING Z (1756575.883 5938257.259 0.000, 1..."
3,4.0,1.0,4.0,0.0,4.0,9.0,1.15,0.93,0.22,9.52,1.00,0.06,72.77,75.41,1.12,0.87,0.31,4.30,30.0,249.399436,"LINESTRING Z (1756572.365 5938266.619 0.000, 1..."
4,5.0,1.0,5.0,0.0,5.0,9.0,1.12,0.96,0.17,7.49,1.03,0.06,74.66,76.41,1.10,0.89,0.27,3.82,40.0,249.399436,"LINESTRING Z (1756568.846 5938275.980 0.000, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,101.0,1.0,101.0,0.0,101.0,10.0,0.18,0.25,0.20,8.97,0.24,0.06,17.58,35.85,0.23,0.24,0.27,3.97,1000.0,219.472459,"LINESTRING Z (1756230.819 5939172.612 0.000, 1..."
101,102.0,1.0,102.0,0.0,102.0,10.0,0.19,0.26,0.21,9.19,0.27,0.06,19.81,37.40,0.25,0.25,0.28,4.14,1010.0,219.472459,"LINESTRING Z (1756223.100 5939178.969 0.000, 1..."
102,103.0,1.0,103.0,0.0,103.0,10.0,0.21,0.24,0.24,10.82,0.32,0.06,23.33,42.46,0.27,0.23,0.32,4.67,1020.0,219.472459,"LINESTRING Z (1756215.380 5939185.326 0.000, 1..."
103,104.0,1.0,104.0,0.0,104.0,10.0,0.19,0.24,0.23,10.03,0.27,0.06,19.87,38.64,0.25,0.21,0.31,4.57,1030.0,205.114835,"LINESTRING Z (1756206.493 5939189.834 0.000, 1..."


In [74]:
def get_azimuth(line):
    p1, p2 = line.coords[1:]
    azimuth = math.degrees(math.atan2(p2[0]-p1[0], p2[1]-p1[1]))
    if azimuth < 0:
        azimuth += 360
    return azimuth
lines["Azimuth"] = lines.geometry.apply(get_azimuth)
lines.to_file("TEST/Auckland_LongBay/LongBay_rates.shp")
lines

,ObjectID_1,BaselineID,TransOrder,TransEdit,transectId,ShrCount,LRR,LR2,LCI,LSE,EPR,EPRunc,NSM,SCE,WLR,WR2,WCI,WSE,TCD,Azimuth,geometry
0,1.0,1.0,1.0,0.0,1.0,7.0,1.14,0.90,0.34,10.54,0.96,0.07,57.64,70.49,1.01,0.60,0.74,7.08,0.0,249.399436,"LINESTRING Z (1756582.920 5938238.537 0.000, 1..."
1,2.0,1.0,2.0,0.0,2.0,8.0,1.10,0.92,0.26,10.67,0.88,0.06,63.77,73.77,1.07,0.78,0.45,5.98,10.0,249.399436,"LINESTRING Z (1756579.402 5938247.898 0.000, 1..."
2,3.0,1.0,3.0,0.0,3.0,8.0,1.15,0.95,0.21,8.61,0.97,0.06,70.11,74.41,1.12,0.86,0.37,4.82,20.0,249.399436,"LINESTRING Z (1756575.883 5938257.259 0.000, 1..."
3,4.0,1.0,4.0,0.0,4.0,9.0,1.15,0.93,0.22,9.52,1.00,0.06,72.77,75.41,1.12,0.87,0.31,4.30,30.0,249.399436,"LINESTRING Z (1756572.365 5938266.619 0.000, 1..."
4,5.0,1.0,5.0,0.0,5.0,9.0,1.12,0.96,0.17,7.49,1.03,0.06,74.66,76.41,1.10,0.89,0.27,3.82,40.0,249.399436,"LINESTRING Z (1756568.846 5938275.980 0.000, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,101.0,1.0,101.0,0.0,101.0,10.0,0.18,0.25,0.20,8.97,0.24,0.06,17.58,35.85,0.23,0.24,0.27,3.97,1000.0,219.472459,"LINESTRING Z (1756230.819 5939172.612 0.000, 1..."
101,102.0,1.0,102.0,0.0,102.0,10.0,0.19,0.26,0.21,9.19,0.27,0.06,19.81,37.40,0.25,0.25,0.28,4.14,1010.0,219.472459,"LINESTRING Z (1756223.100 5939178.969 0.000, 1..."
102,103.0,1.0,103.0,0.0,103.0,10.0,0.21,0.24,0.24,10.82,0.32,0.06,23.33,42.46,0.27,0.23,0.32,4.67,1020.0,219.472459,"LINESTRING Z (1756215.380 5939185.326 0.000, 1..."
103,104.0,1.0,104.0,0.0,104.0,10.0,0.19,0.24,0.23,10.03,0.27,0.06,19.87,38.64,0.25,0.21,0.31,4.57,1030.0,205.114835,"LINESTRING Z (1756206.493 5939189.834 0.000, 1..."


In [75]:
gdf.Year.describe()

count    1027.000000
mean     1995.465433
std        26.236433
min      1950.000000
25%      1973.000000
50%      2011.000000
75%      2022.000000
max      2023.000000
Name: Year, dtype: float64

In [76]:
gdf.Distance.describe()

count    1027.000000
mean      -27.050312
std        11.596512
min       -98.700000
25%       -28.150000
50%       -24.790000
75%       -21.595000
max       -13.560000
Name: Distance, dtype: float64

In [77]:
def get_transect_metadata(transect_lines_shapefile: str):
    lines = gpd.read_file(transect_lines_shapefile)
    if "TransectID" in lines.columns:
        lines = lines.set_index("TransectID").sort_index()
    else:
        lines = lines.set_index("TransOrder").sort_index()
    lines["dist_to_neighbour"] = lines.distance(lines.shift(-1))
    breakpoints = lines.dist_to_neighbour[lines.dist_to_neighbour > 105]
    lines["group"] = pd.Series(range(len(breakpoints)), index=breakpoints.index)
    lines["group"] = lines.group.bfill().fillna(len(breakpoints)).astype(int)
    metadata = lines[["Azimuth", "group"]].to_dict(orient="index")
    return metadata

transect_metadata = get_transect_metadata(f"TEST/Auckland_LongBay/LongBay_rates.shp")
transect_metadata

{1.0: {'Azimuth': 249.39943620766252, 'group': 0},
 2.0: {'Azimuth': 249.39943589302055, 'group': 0},
 3.0: {'Azimuth': 249.39943589302055, 'group': 0},
 4.0: {'Azimuth': 249.39943589302055, 'group': 0},
 5.0: {'Azimuth': 249.39943589302055, 'group': 0},
 6.0: {'Azimuth': 249.39943589302055, 'group': 0},
 7.0: {'Azimuth': 249.39943589302055, 'group': 0},
 8.0: {'Azimuth': 249.39943589302055, 'group': 0},
 9.0: {'Azimuth': 249.39943589302055, 'group': 0},
 10.0: {'Azimuth': 249.39943589302055, 'group': 0},
 11.0: {'Azimuth': 248.67560231118233, 'group': 0},
 12.0: {'Azimuth': 248.67560231118233, 'group': 0},
 13.0: {'Azimuth': 248.67560328151876, 'group': 0},
 14.0: {'Azimuth': 248.67560231118233, 'group': 0},
 15.0: {'Azimuth': 248.67560231118233, 'group': 0},
 16.0: {'Azimuth': 248.67560231118233, 'group': 0},
 17.0: {'Azimuth': 248.67560231118233, 'group': 0},
 18.0: {'Azimuth': 248.67560328151876, 'group': 0},
 19.0: {'Azimuth': 248.67560231118233, 'group': 0},
 20.0: {'Azimuth': 24

In [78]:
linear_models = fit(gdf, transect_metadata)
linear_models

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse
0,1.0,1.137493,-287.113293,0,0.899979,7.719117,81.226923,9.012598
1,2.0,1.094596,-271.071182,0,0.918585,7.778484,86.694893,9.311009
2,3.0,1.146533,-276.724950,0,0.950353,5.837518,56.063658,7.487567
3,4.0,1.144073,-276.507210,0,0.932981,5.839418,70.903340,8.420412
4,5.0,1.116530,-269.600165,0,0.954922,5.013590,44.378314,6.661705
...,...,...,...,...,...,...,...,...
100,101.0,0.174220,-61.579520,0,0.243565,6.791799,64.534554,8.033340
101,102.0,0.186240,-63.207372,0,0.259564,6.996329,67.737057,8.230253
102,103.0,0.208432,-69.212941,0,0.240615,8.333542,93.865278,9.688409
103,104.0,0.192922,-65.105831,0,0.239869,7.921248,80.745771,8.985865


In [64]:
linear_models = fit(gdf, transect_metadata)
rolled_slopes = linear_models.groupby("group").slope.rolling(10, min_periods=1).mean().dropna().reset_index(level=0)
linear_models.slope = rolled_slopes.slope
linear_models.dropna(inplace=True)
linear_models

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse
0,1.0,1.137493,-287.113293,0,0.899979,7.719117,81.226923,9.012598
1,2.0,1.116044,-271.071182,0,0.918585,7.778484,86.694893,9.311009
2,3.0,1.126207,-276.724950,0,0.950353,5.837518,56.063658,7.487567
3,4.0,1.130674,-276.507210,0,0.932981,5.839418,70.903340,8.420412
4,5.0,1.127845,-269.600165,0,0.954922,5.013590,44.378314,6.661705
...,...,...,...,...,...,...,...,...
100,101.0,0.164541,-61.579520,0,0.243565,6.791799,64.534554,8.033340
101,102.0,0.170545,-63.207372,0,0.259564,6.996329,67.737057,8.230253
102,103.0,0.177131,-69.212941,0,0.240615,8.333542,93.865278,9.688409
103,104.0,0.181155,-65.105831,0,0.239869,7.921248,80.745771,8.985865


In [79]:
#Changed coordinate function by making old_x and old_y negative 
def calculate_new_coordinates(old_x, old_y, bearing, distance):
    bearing_radians = math.radians(bearing)
    new_x = old_x + (distance * math.sin(bearing_radians))
    new_y = old_y + (distance * math.cos(bearing_radians))
    point = Point(new_x, new_y)
    assert not point.is_empty
    return point

#Removed other model equations and changed Azimuth addtion from 180 to 360 deg
def predict(
    df: pd.DataFrame,
    linear_models: pd.DataFrame,
    transect_metadata: dict,
):
    """_summary_

    Args:
        df (pd.DataFrame): dataframe with columns: TransectID, Date, Distance, YearsSinceBase
        linear_models (pd.DataFrame): dataframe with columns: TransectID, slope, intercept
        transect_metadata (dict): dict lookup of TransectID to Azimuth & group
        
    Returns:
        pd.DataFrame: resulting prediction points for the year 2100
    """
    results = []
    for i, row in linear_models.iterrows():
        transect_ID = row.TransectID
        transect_df = df[df.TransectID == transect_ID]
        latest_row = transect_df[transect_df.Date == transect_df["Date"].max()].iloc[0]
        future_year = int(row.get("FUTURE_YEAR", FUTURE_YEAR))
        result = row.to_dict()
        result.update({
            "TransectID": transect_ID,
            "BaselineID": latest_row.BaselineID,
            "group": row.group,
            "Year": future_year,
            "ocean_point": calculate_new_coordinates(
                latest_row.geometry.x,
                latest_row.geometry.y,
                transect_metadata[transect_ID]["Azimuth"] + 180,
                500,
            ),
        })
        
        model = "linear"
        slope = row.slope
        intercept = row.intercept

        predicted_distance = slope * (future_year - 1800) + intercept
        distance_difference = latest_row.Distance - predicted_distance
        result[f"{model}_model_point"] = calculate_new_coordinates(
            latest_row.geometry.x,
            latest_row.geometry.y,
            transect_metadata[transect_ID]["Azimuth"],
            distance_difference,
        )
        result[f"{model}_model_predicted_distance"] = predicted_distance
        result[f"{model}_model_distance"] = distance_difference
        results.append(result)
    results = gpd.GeoDataFrame(results)
    return results

In [80]:
results = predict(gdf, linear_models, transect_metadata)
results

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse,BaselineID,Year,ocean_point,linear_model_point,linear_model_predicted_distance,linear_model_distance
0,1.0,1.137493,-287.113293,0.0,0.899979,7.719117,81.226923,9.012598,1.0,2100,POINT (1757015.0880949597 5938400.98308736),POINT (1756633.5932574873 5938257.584320819),54.134461,-92.444461
1,2.0,1.094596,-271.071182,0.0,0.918585,7.778484,86.694893,9.311009,1.0,2100,POINT (1757018.3019194454 5938412.874244093),POINT (1756633.0471306564 5938268.062160125),57.307659,-88.427659
2,3.0,1.146533,-276.724950,0.0,0.950353,5.837518,56.063658,7.487567,1.0,2100,POINT (1757023.6913963815 5938425.583196558),POINT (1756638.8178731413 5938280.914425172),67.234970,-88.834970
3,4.0,1.144073,-276.507210,0.0,0.932981,5.839418,70.903340,8.420412,1.0,2100,POINT (1757019.7895928912 5938434.799681907),POINT (1756634.812859649 5938290.092115272),66.714709,-88.724709
4,5.0,1.116530,-269.600165,0.0,0.954922,5.013590,44.378314,6.661705,1.0,2100,POINT (1757015.660730892 5938443.930819021),POINT (1756630.0232727972 5938298.974894817),65.358849,-88.018849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,101.0,0.174220,-61.579520,0.0,0.243565,6.791799,64.534554,8.033340,1.0,2100,POINT (1756532.6036785953 5939539.063643764),POINT (1756224.90012968 5939165.4236136405),-9.313414,-15.966586
101,102.0,0.186240,-63.207372,0.0,0.259564,6.996329,67.737057,8.230253,1.0,2100,POINT (1756524.1940497549 5939544.58246269),POINT (1756218.4345011374 5939173.303004397),-7.335402,-19.024598
102,103.0,0.208432,-69.212941,0.0,0.240615,8.333542,93.865278,9.688409,1.0,2100,POINT (1756515.3581770945 5939549.583699825),POINT (1756211.131951556 5939180.166133875),-6.683407,-21.436593
103,104.0,0.192922,-65.105831,0.0,0.239869,7.921248,80.745771,8.985865,1.0,2100,POINT (1756407.0566537792 5939617.702309341),POINT (1756203.4263998063 5939183.291098633),-7.229091,-20.230909


In [81]:
results.set_geometry("linear_model_point", inplace=True, crs=2193)
results

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse,BaselineID,Year,ocean_point,linear_model_point,linear_model_predicted_distance,linear_model_distance
0,1.0,1.137493,-287.113293,0.0,0.899979,7.719117,81.226923,9.012598,1.0,2100,POINT (1757015.0880949597 5938400.98308736),POINT (1756633.593 5938257.584),54.134461,-92.444461
1,2.0,1.094596,-271.071182,0.0,0.918585,7.778484,86.694893,9.311009,1.0,2100,POINT (1757018.3019194454 5938412.874244093),POINT (1756633.047 5938268.062),57.307659,-88.427659
2,3.0,1.146533,-276.724950,0.0,0.950353,5.837518,56.063658,7.487567,1.0,2100,POINT (1757023.6913963815 5938425.583196558),POINT (1756638.818 5938280.914),67.234970,-88.834970
3,4.0,1.144073,-276.507210,0.0,0.932981,5.839418,70.903340,8.420412,1.0,2100,POINT (1757019.7895928912 5938434.799681907),POINT (1756634.813 5938290.092),66.714709,-88.724709
4,5.0,1.116530,-269.600165,0.0,0.954922,5.013590,44.378314,6.661705,1.0,2100,POINT (1757015.660730892 5938443.930819021),POINT (1756630.023 5938298.975),65.358849,-88.018849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,101.0,0.174220,-61.579520,0.0,0.243565,6.791799,64.534554,8.033340,1.0,2100,POINT (1756532.6036785953 5939539.063643764),POINT (1756224.900 5939165.424),-9.313414,-15.966586
101,102.0,0.186240,-63.207372,0.0,0.259564,6.996329,67.737057,8.230253,1.0,2100,POINT (1756524.1940497549 5939544.58246269),POINT (1756218.435 5939173.303),-7.335402,-19.024598
102,103.0,0.208432,-69.212941,0.0,0.240615,8.333542,93.865278,9.688409,1.0,2100,POINT (1756515.3581770945 5939549.583699825),POINT (1756211.132 5939180.166),-6.683407,-21.436593
103,104.0,0.192922,-65.105831,0.0,0.239869,7.921248,80.745771,8.985865,1.0,2100,POINT (1756407.0566537792 5939617.702309341),POINT (1756203.426 5939183.291),-7.229091,-20.230909


In [82]:
poly = prediction_results_to_polygon(results)
output_shapefile = "Z:/DSAS/Region/Auckland/LongBay/LongBay_projection_output.shp"
poly.to_file(output_shapefile, driver="ESRI Shapefile")

In [83]:
m = poly.explore(tiles="Esri.WorldImagery")
gpd.GeoDataFrame(results.drop(columns=["ocean_point", "linear_model_point"]), geometry=results.linear_model_point).explore(m=m)
gdf.explore("Year", legend=True, m=m)